# Day 10 - Sorting adapters

* https://adventofcode.com/2020/day/10

Eric Wastle is trying to trick us again with a convoluted description; all we have to do is *sort* the adapters. That's because, to be able to use all adapters, can only put any given adapter between adapters that are lower on the one end, and higher on the other.

Once sorted, we can keep a count of their differences; a `Counter()` can do that just fine.

To keep track of the preceding adapter (to calculate the jolt differential), I used the `pairwise` recipe from the [`itertools` recipes section](https://docs.python.org/3/library/itertools.html#itertools-recipes); that's a fixed window iterator, really.

In [1]:
from collections import Counter
from collections.abc import Sequence
from itertools import chain, tee

def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

def parse_adapters(input_: str) -> Sequence[int]:
    return [int(v) for v in input_.split()]

def all_adapters(adapters: Sequence[int]) -> int:
    # (0, adapter1), (adapter1, adapter2), ...
    paired = pairwise(chain([0], sorted(adapters)))
    differences = Counter(adapter - prev for prev, adapter in paired)
    return differences[1] * (differences[3] + 1)  # the device itself counts too

tests = {
    "16 10 15 5 1 11 7 19 6 12 4": 7 * 5,
    (
        "28 33 18 42 31 14 46 20 48 47 24 23 49 45 "
        "19 38 39 11 1 32 25 35 8 17 7 9 4 2 34 10 3"
    ): 22 * 10,
}
for test, expected in tests.items():
    assert all_adapters(parse_adapters(test)) == expected

In [2]:
import aocd
adapters = parse_adapters(aocd.get_data(day=10, year=2020))

In [3]:
print("Part 1:", all_adapters(adapters))

Part 1: 2277


## Part 2 - Calculating the number of combinations.

Finally, we are entering more advanced territories in this years AoC. To calculate the number of combinations, there are two approaches we can take:

- Recursion, the number of combinations of a given adapter with jolt level *J*, is the sum of all adapters with jolt levels *J + 1*, *J + 2* and *J + 3* that exist. Eventually you'll reach the last adapter (your device), so you reach the base case with a value of `1`. You'd be smart to use memoization there, as those *J + 2 and *J + 3* adapters are often also reachable from the *J + 1* and *J + 2* positions, you wouldn't want to have to calculate those numbers again!

  The disadvantage here is that we need, even with memoization, `len(adapters)` spaces in our cache to reach the end and so finally sum up the total number of combinations. And while you could unroll the recursion and use an explicit stack instead of the call stack to track you progress, that's still a lot of work.

- Inverting the calculation, and use *dynamic programming*. If the number of combinations for an adapter of jolt level *J* depends on the values for the adapters of *J* + 1, *J* + 2 and *J* + 3, could we first calculate the number of combinations for the *last* 3 adapters in the chain, and work our way back from there? That way we would only need enough memory to remember the number of combinations of 3 adapters, at most, throughout!

  Note that the puzzle explicitly tells us that the last element, your device, is *always* at *J + 3* from the adapter before it. So the _base case_ here is that for the one-but-last adapter the number of combinations is never more than 1 *either* (it can only connect directly to your device), and the adapter at the second-last position, there is *also* just one option, you can't skip the one-but-last adapter.

To implement the latter option, I chose a `deque` to represent the number of combinations you can make with adapters (if they exist), at jolt levels `+1`, `+2` and `+3`, and then just prepend the value for our adapter to that point, and rotate in zeros to the start depending on how many jolt levels need to be skipped to reach the next last adapter. Because the `deque` is configured to only hold 3 values, rotating in zeros neatly drops off values from the end as adapters at higher jolt levels disappear from consideration:

In [4]:
from collections import deque

def calc_combinations(adapters: Sequence[int]) -> int:
    # start with [1], and max() + 3, to represent the device at the end
    combos = deque([1], maxlen=3)
    next_ = max(adapters) + 3
    # (device, adapters[-1]), (adapters[-1], adapters[-2]), ...
    with_next = pairwise(chain([next_], sorted(adapters, reverse=True)))
    for next_, adapter in with_next:
        combos.extendleft([0] * (next_ - adapter - 1))  # automatically drops values on the opposite end
        combos.appendleft(sum(combos))  # add up all combinations of adapters we can reach
    return sum(combos)


tests = {
    "16 10 15 5 1 11 7 19 6 12 4": 8,
    (
        "28 33 18 42 31 14 46 20 48 47 24 23 49 45 "
        "19 38 39 11 1 32 25 35 8 17 7 9 4 2 34 10 3"
    ): 19208,
}
for test, expected in tests.items():
    assert calc_combinations(parse_adapters(test)) == expected

In [5]:
print("Part 2:", calc_combinations(adapters))

Part 2: 37024595836928
